In [ ]:
pip install boto3 minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.6 MB/s eta 0:00:00


Import all necessary libraries.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import boto3
from minio import Minio

In [ ]:
minio_client = Minio('172.25.64.1:9000',
                     'dev',
                     'password',
                     secure=False)  # Set to True if using HTTPS


In [ ]:
buckets = minio_client.list_buckets()
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

KeyboardInterrupt: 

Load CSV file into dataframe.

In [ ]:
df = pd.read_csv("BTC-USD.csv")

Remove empty values and show shape of dataframe.

In [ ]:
df.dropna
df.shape

(3505, 7)

Show first five rows of dataframe.

In [ ]:
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100


Show some statistical data of the dataframe.

In [ ]:
df.describe()

,Open,High,Low,Close,Adj Close,Volume
count,3505.000000,3505.000000,3505.000000,3505.000000,3505.000000,3.505000e+03
mean,15890.348935,16262.529884,15492.619787,15907.486656,15907.486656,1.712677e+10
std,17619.066819,18043.450106,17150.136977,17634.128970,17634.128970,1.928778e+10
min,176.897003,211.731003,171.509995,178.102997,178.102997,5.914570e+06
25%,1007.650024,1040.469971,983.221008,1011.799988,1011.799988,2.093120e+08
50%,8671.212891,8843.366211,8395.120117,8672.455078,8672.455078,1.276360e+10
75%,26760.851562,27150.976562,26389.300781,26784.078125,26784.078125,2.763576e+10
max,73079.375000,73750.070312,71334.093750,73083.500000,73083.500000,3.509679e+11


Show datatypes in dataframe.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3505 entries, 0 to 3504
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3505 non-null   object 
 1   Open       3505 non-null   float64
 2   High       3505 non-null   float64
 3   Low        3505 non-null   float64
 4   Close      3505 non-null   float64
 5   Adj Close  3505 non-null   float64
 6   Volume     3505 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 191.8+ KB


Show unique values in dataframe.

In [ ]:
df.nunique()

Date         3505
Open         3501
High         3502
Low          3503
Close        3502
Adj Close    3502
Volume       3505
dtype: int64

In [ ]:
input_df = df[['Open', 'High', 'Low', 'Close']]
input_df

,Open,High,Low,Close
0,465.864014,468.174011,452.421997,457.334015
1,456.859985,456.859985,413.104004,424.440002
2,424.102997,427.834991,384.532013,394.795990
3,394.673004,423.295990,389.882996,408.903992
4,408.084991,412.425995,393.181000,398.821014
...,...,...,...,...
3500,63831.847656,64486.363281,59768.585938,61276.691406
3501,61275.316406,64125.687500,60833.480469,63512.753906
3502,63510.750000,65481.597656,59651.390625,63843.570312
3503,63851.101562,65442.457031,63172.402344,64994.441406


Import necessary libraries.

Define x and y values with dataframe.

In [ ]:
x = input_df[['Open', 'High', 'Low']]
y = input_df['Close']

Split the dataframe in train and test sets.

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

Show shape of train and test sets.

In [ ]:
print("X_train:",x_train.shape)
print("X_test:",x_test.shape)
print("Y_train:",y_train.shape)
print("Y_test:",y_test.shape)

X_train: (2804, 3)
X_test: (701, 3)
Y_train: (2804,)
Y_test: (701,)


In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict([[408.084991, 412.425995, 393.181000]])
y_pred

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([400.04561193])